In [15]:
#!pip install ipynb

In [16]:
import numpy as np
import threading
import random
from protocol_tests import test_all
from qiskit_aer import AerSimulator
from qiskit import transpile, QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from threading import Thread, Lock
from globals import *



import sys
try:
    del sys.modules['ipynb.fs.full.weak_global_coin']
except KeyError:
    pass

import ipynb.fs.full.weak_global_coin as QCF

try:
    del sys.modules['globals']
except KeyError:
    pass

In [17]:
broadcasted_messages = []
first_to_decide = None

QUESTION_MARK = "?"
WAITING_MESSAGE = "waiting"
HALF_PLUS_ONE = int(np.floor(n/2)) + 1
MAX_ALIVE_PROCESSES = n - t

broadcasting_lock = Lock()
decision_lock = Lock()

In [18]:
class Process:
    def __init__(self, id, input_val, faulty=False):
        self.id = id
        self.input_val = input_val
        self.output = None
        self.faulty = faulty
        self.non_faulty = not faulty
        self.round_messages = {}  # holds messages received in a round
        self.decision_epoch = None

class Message:
    def __init__(self, sender_id, receivers, epoch, round, message):
        self.sender_id = sender_id
        self.receivers = receivers
        self.epoch = epoch
        self.round = round
        self.message = message
        self.read = {pid: False for pid in receivers}

In [19]:
def broadcast(process_id, epoch, round, message):
    new_msg = Message(process_id, list(range(n)), epoch, round, message)
    broadcasting_lock.acquire()
    broadcasted_messages.append(new_msg)
    broadcasting_lock.release()

def waiting_condition(num_received_messages, round):
    if round in [1, 2]:
        actual_alive = sum([1 for p in processes if p.non_faulty])
        return num_received_messages < actual_alive
    elif round == 3:
        return num_received_messages < MAX_ALIVE_PROCESSES

def receive(process, epoch, round, required_val=None):
    num_received_messages = 0
    while waiting_condition(num_received_messages, round):
        broadcasting_lock.acquire()
        for msg in broadcasted_messages:
            if msg.epoch == epoch and msg.round == round and process.id in msg.receivers and not msg.read[process.id]:
                if round == 3:
                    assert msg.message == required_val
                count = process.round_messages.get(msg.message, 0)
                process.round_messages[msg.message] = count + 1
                msg.read[process.id] = True
                num_received_messages += 1
        broadcasting_lock.release()


def get_majority_value(process):
    for value, count in process.round_messages.items():
        if count >= HALF_PLUS_ONE:
            return value
    return QUESTION_MARK

def get_most_frequent_val(process):
    if not process.round_messages:
        return QUESTION_MARK, 0
    most_freq = max(process.round_messages, key=process.round_messages.get)
    count = process.round_messages[most_freq]
    if most_freq == QUESTION_MARK:
        process.round_messages.pop(most_freq)
        if process.round_messages:
            most_freq = max(process.round_messages, key=process.round_messages.get)
            count = process.round_messages[most_freq]
        else:
            return QUESTION_MARK, 0
    return most_freq, count

In [ ]:
def agreement(process):
    current = process.input_val
    next_decide = False
    epoch = 0
    while True:
        epoch += 1

        if process.faulty:
            print(f"[Faulty Process {process.id}] Injecting bad data")
            # 🛠️ Modified: faulty nodes now still broadcast, but with garbage data
            broadcast(process.id, epoch, 1, random.choice(["0", "1", "?", "X"]))
        else:
            broadcast(process.id, epoch, 1, current)

        if not next_decide:
            receive(process, epoch, 1)
            current = get_majority_value(process)
        process.round_messages.clear()

        if process.faulty:
            broadcast(process.id, epoch, 2, random.choice(["0", "1", "?", "X"]))
        else:
            broadcast(process.id, epoch, 2, current)

        if not next_decide:
            receive(process, epoch, 2)
            answer, number = get_most_frequent_val(process)
        process.round_messages.clear()

        broadcast(process.id, epoch, 3, WAITING_MESSAGE)
        if not next_decide:
            receive(process, epoch, 3, WAITING_MESSAGE)
        process.round_messages.clear()

        try:
            result = QCF.quantum_coin_flip(processes, process, epoch)
            if isinstance(result, tuple):  # honest process: (leader, coin)
                leader, coin = result
            else:  # faulty process: just coin
                coin = result
        except Exception as e:
            print(f"[Error in coin flip for Process {process.id}] {e}")
            coin = random.randint(0, 1)


        if next_decide:
            break

        if number >= HALF_PLUS_ONE:
            current = answer
            next_decide = True
            process.decision_epoch = epoch
            decision_lock.acquire()
            global first_to_decide
            if first_to_decide is None:
                first_to_decide = process.id
            decision_lock.release()
        elif number >= 1:
            current = answer
        else:
            current = coin

    process.output = current
    return current


In [21]:
inputs = [random.randint(0, 1) for _ in range(n)]
faulty_ids = set(random.sample(range(n), t))

processes = [Process(id=i, input_val=inputs[i], faulty=(i in faulty_ids)) for i in range(n)]

threads = []
for pr in processes:
    thr = Thread(target=agreement, args=(pr,))
    thr.start()
    threads.append(thr)

for thr in threads:
    thr.join()

# --- Results ---
print("\n--- Final Decisions ---")
for pr in processes:
    print(f"Process {pr.id} | Output: {pr.output} | Faulty: {pr.faulty} | Decision Epoch: {pr.decision_epoch}")

print(f"\nFirst to decide: Process {first_to_decide}")


[Faulty Process 0] Injecting bad data
[Faulty Process 0] Injecting bad data


KeyboardInterrupt: 

In [ ]:
test_all(processes, first_to_decide, broadcasted_messages)